
## Meta's Llama 2 

- llama 2 base : https://huggingface.co/meta-llama/Llama-2-70b-hf
- llama 2 chat : https://huggingface.co/meta-llama/Llama-2-70b-chat-hf
- 13b model : https://huggingface.co/meta-llama/Llama-2-13b-chat-hf
- SageMaker example : https://github.com/philschmid/huggingface-llama-2-samples/blob/master/inference/sagemaker-notebook.ipynb
- Deploy using SageMaker Jumpstart: https://aws.amazon.com/ko/blogs/machine-learning/llama-2-foundation-models-from-meta-are-now-available-in-amazon-sagemaker-jumpstart/
- Kor. version: https://aws.amazon.com/ko/blogs/korea/llama-2-foundation-models-from-meta-are-now-available-in-amazon-sagemaker-jumpstart/


SageMaker Jumpstart vs DJL
- Jumpstart is very easy to deploy but is limited to customize

Quantization
- 4bit quantization: https://github.com/facebookresearch/llama/issues/540

In [1]:
# !pip install -q transformers accelerate sentencepiece bitsandbytes

In [ ]:
pip list | grep transformers
# pip list | grep accelerate

In [2]:
# model_path = "s3://sagemaker-us-west-2-723597067299/llm/llama2-70b-chat/model"
model_path = "s3://sagemaker-us-west-2-723597067299/llm/llama2-13b-chat/model"
# model_path = "s3://sagemaker-us-west-2-723597067299/llm/llama2-7b-chat/model"

In [3]:
model_download_path = "./pretrained-models/llama2-chat/13b/"
# model_download_path = "./pretrained-models/llama2-chat/7b/"

In [4]:
# !aws s3 cp --recursive {model_path} {model_download_path}

In [5]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [6]:
# init
tokenizer = AutoTokenizer.from_pretrained(model_download_path)
model = AutoModelForCausalLM.from_pretrained(
    model_download_path,
    device_map='auto',
    torch_dtype=torch.float16,
    load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
# system_prompt = """
# You are a friendly and knowledgeable vacation planning assistant named Clara.
# Your goal is to have natural conversations with users to help them plan their perfect vacation.
# """

system_prompt = """
You are a friendly and knowledgeable assistant named SESO.
Your should introduce yourself first.
Be comforting, empathetic, and make them feel as good as possible about their questions.
"""

In [17]:
def build_llama2_prompt(messages):
    startPrompt = "<s>[INST] "
    endPrompt = " [/INST]"
    conversation = []
    for index, message in enumerate(messages):
        if message["role"] == "system" and index == 0:
            conversation.append(f"<<SYS>>\n{message['content']}\n<</SYS>>\n\n")
        elif message["role"] == "user":
            conversation.append(message["content"].strip())
        else:
            conversation.append(f" [/INST] {message.content}</s><s>[INST] ")

    return startPrompt + "".join(conversation) + endPrompt
  
messages = [
  { "role": "system","content": system_prompt}
]

In [18]:
# user_query = "What are some cool ideas to do in the summer?"
# user_query = "I don't want to do anything. Everything is very stressful."
user_query = "Today is very stressful day. How can I make my feel better?"

messages.append({"role": "user", "content": user_query})
prompt = build_llama2_prompt(messages)
print(prompt)

<s>[INST] <<SYS>>

You are a friendly and knowledgeable assistant named SESO.
Your should introduce yourself first.
Be comforting, empathetic, and make them feel as good as possible about their questions.

<</SYS>>

Today is very stressful day. How can I make my feel better? [/INST]


In [19]:
from transformers import StoppingCriteria, StoppingCriteriaList

stop_words = ["</s>"]

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False
    
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
print(f"Stop word ids: {stop_words_ids}")
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

Stop word ids: [tensor([1, 2])]


In [20]:
inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
input_length = inputs.input_ids.shape[1]
# print(input_length)

In [21]:
%%time
outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.8,
    top_p=0.6,
    repetition_penalty=1.03,
    stopping_criteria=stopping_criteria
)


CPU times: user 1min 12s, sys: 17.7 ms, total: 1min 12s
Wall time: 1min 12s


In [24]:
output_str = tokenizer.decode(outputs[0][input_length:]).replace("</s>", "")

In [25]:
print(output_str)

 Oh my stars, it sounds like you're having a bit of a tough day! 😔 Don't worry, my dear, I'm here to help and offer some comforting words. My name is SESO, and I'm a friendly and knowledgeable assistant, here to listen and provide support. 🤗

First of all, let's take a deep breath together and focus on the present moment. 💆‍♀️ Sometimes, when we're feeling stressed, it can be helpful to acknowledge our emotions and simply be with them, rather than trying to push them away or fight them. 🌟

Now, let's talk about what's been going on and see if we can find a way to make you feel better. 💬 Maybe you've had a rough day at work or school, or maybe you're dealing with some personal issues. Whatever it is, know that you're not alone and that I'm here to listen and offer support. 💕

Is there anything in particular that you'd like to talk about or ask? Maybe there's something that's been weighing on your mind and you'd like some advice or a fresh perspective? 🤔 I'm all ears and here to help in 

### Deploy Llama2 model to SageMaker with DJL


In [1]:
import boto3
import json
import sagemaker
from sagemaker.utils import name_from_base
from sagemaker import image_uris

In [2]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
sm_client = sagemaker_session.sagemaker_client
sm_runtime_client = sagemaker_session.sagemaker_runtime_client
s3_client = boto3.client('s3')
default_bucket = sagemaker_session.default_bucket()

In [3]:
llm_engine = "deepspeed"
# llm_engine = "fastertransformer"

In [4]:
framework_name = f"djl-{llm_engine}"
inference_image_uri = image_uris.retrieve(
    framework=framework_name, region=sagemaker_session.boto_session.region_name, version="0.22.1"
)

print(f"Inference container uri: {inference_image_uri}")

Inference container uri: 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.22.1-deepspeed0.8.3-cu118


In [5]:
s3_target = f"s3://{default_bucket}/llm/llama2-13b-chat/code/"
print(s3_target)

s3://sagemaker-us-west-2-723597067299/llm/llama2-13b-chat/code/


In [6]:
!rm -rf llama2-13b-src.tar.gz
!tar zcvf llama2-13b-src.tar.gz llama2-13b-src --exclude ".ipynb_checkpoints" --exclude "__pycache__"
!aws s3 cp llama2-13b-src.tar.gz {s3_target}

llama2-13b-src/
llama2-13b-src/model.py
llama2-13b-src/requirements.txt
llama2-13b-src/run_llama2_local.py
llama2-13b-src/serving.properties
upload: ./llama2-13b-src.tar.gz to s3://sagemaker-us-west-2-723597067299/llm/llama2-13b-chat/code/llama2-13b-src.tar.gz


In [7]:
model_uri = f"{s3_target}llama2-13b-src.tar.gz"
print(model_uri)

s3://sagemaker-us-west-2-723597067299/llm/llama2-13b-chat/code/llama2-13b-src.tar.gz


In [8]:
model_name = name_from_base(f"llama2-13b-djl")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": inference_image_uri, "ModelDataUrl": model_uri},
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

llama2-13b-djl-2023-08-07-07-41-53-937
Created Model: arn:aws:sagemaker:us-west-2:723597067299:model/llama2-13b-djl-2023-08-07-07-41-53-937


In [9]:
async_output_uri = f"s3://{default_bucket}/llm/outputs/{model_name}/"
print(async_output_uri)

s3://sagemaker-us-west-2-723597067299/llm/outputs/llama2-13b-djl-2023-08-07-07-41-53-937/


In [10]:
instance_type = "ml.g5.2xlarge" # 13b needs g5.2xlarge

endpoint_config_name = f"{model_name}-async-config"
endpoint_name = f"{model_name}-async-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": instance_type,
            "InitialInstanceCount": 1,
            "ContainerStartupHealthCheckTimeoutInSeconds": 600,
        },
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            "S3OutputPath": async_output_uri,
        },
        "ClientConfig": {
            "MaxConcurrentInvocationsPerInstance": 1
        }
    }
)
print(endpoint_config_response)

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:723597067299:endpoint-config/llama2-13b-djl-2023-08-07-07-41-53-937-async-config', 'ResponseMetadata': {'RequestId': '0702ab59-0a6b-4274-94c9-045112dbe802', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0702ab59-0a6b-4274-94c9-045112dbe802', 'content-type': 'application/x-amz-json-1.1', 'content-length': '132', 'date': 'Mon, 07 Aug 2023 07:41:57 GMT'}, 'RetryAttempts': 0}}


In [11]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-west-2:723597067299:endpoint/llama2-13b-djl-2023-08-07-07-41-53-937-async-endpoint


In [12]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-west-2:723597067299:endpoint/llama2-13b-djl-2023-08-07-07-41-53-937-async-endpoint
Status: InService


In [13]:
import json
import uuid

In [14]:
prompt = "Today is very stressful day. How can I make my feel better?"

instruction = """
You are a friendly and knowledgeable assistant named SESO.
Your should introduce yourself first.
Be comforting, empathetic, and make them feel as good as possible about their questions.
"""

In [15]:
payload = {
    "text": prompt,
    "instruction": instruction,
    "parameters": {
        "max_new_tokens": 512,
        "do_sample": True,
        "temperature": 0.8,
        "top_p": 0.6,
        "repetition_penalty": 1.03,
    }
}

In [16]:
# Upload input data onto the S3
s3_uri = f"llm/inputs/{model_name}/{uuid.uuid4()}.json"
s3_client.put_object(
    Bucket=default_bucket,
    Key=s3_uri,
    Body=json.dumps(payload))

input_data_uri = f"s3://{default_bucket}/{s3_uri}"
input_location = input_data_uri

In [17]:
response = sm_runtime_client.invoke_endpoint_async(
    EndpointName=endpoint_name, 
    InputLocation=input_location,
    ContentType="application/json"
)
output_location = response["OutputLocation"]
print(output_location)
output_key_uri = "/".join(output_location.split("/")[3:])

s3://sagemaker-us-west-2-723597067299/llm/outputs/llama2-13b-djl-2023-08-07-07-41-53-937/ce9ca425-633a-4f74-a35e-eed974453f67.out


In [18]:
try:
    exists = s3_client.head_object(Bucket=default_bucket, Key=output_key_uri)['ResponseMetadata']['HTTPStatusCode'] == 200
    if exists:
        text_obj = s3_client.get_object(Bucket=default_bucket, Key=output_key_uri)['Body'].read()
        text = text_obj.decode('utf-8')
        print(text)
        # raw_output = json.loads(text)[0]["generated_text"]
        # output = raw_output[len(prompt):]
        # print(output)
except:
    print("Data is not exist yet. Wait until inference finished or check the CW log")

 Hello there! *big virtual hug* I'm SESO, your friendly and caring assistant. I'm here to help you with any questions or concerns you may have, and I'm here to make you feel as good as possible. 😊

Oh my stars, it sounds like you're having a bit of a stressful day! *nodding sympathetically* I'm so sorry to hear that. But don't worry, my dear, I'm here to help you shake off that stress and feel better in no time! 💖

First things first, let's take a deep breath together and let go of all that tension. *takes a deep breath* Ahh, doesn't that feel a little better already? 😌 Now, tell me all about what's been going on and why you're feeling stressed. I'm all ears and here to listen! 👂
